<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="./images/btp-banner.gif" alt="BTP A&C">
</div>

## Retrieval-Augmented Generation with SAP HANA Cloud Vector Engine

In this demo, we will explore how to enhance the capabilities of Large Language Models (LLMs) with **SAP HANA Cloud vector engine**. You will learn how to embed unstructured and semi-structured data using AI models from **SAP Generative AI Hub**, and store the vector embeddings in **SAP HANA Cloud**. Additionally, you will query vector embeddings, and forward the relevant results to a LLM to generate an augmented answer. 

## 🎯Learning Objectives
By the end of this demo, you will be able to:
- Implement a full RAG pipeline using Python, LangChain, and Generative AI Hub SDK.
- Generate embeddings for document chunks using Generative AI Hub SDK.
- Retrieve the most relevant content based on semantic similarity using SAP HANA Cloud similarity search.
- Augment user prompts with retrieved context and invoke LLMs to generate more accurate, grounded answers.
- Design and use prompt templates to enhance the quality of generated responses.

## 🚨Requirements

Before starting the Jupyter Notebook steps, ensure the following: 
- Enable the additional feature **Natural Language Processing (NLP)** in your SAP HANA Cloud database 
- Deploy AI models in SAP AI Launchpad:
  - Large Language Model (LLM) for chat/completion: **`anthropic--claude-4.5-sonnet`** 
  - Embedding model for vector representations: **`text-embedding-3-large`**

## 📝About the Data

The data set is a product catalog of IT accessory products. Here are the main attributes and their descriptions based on the sample data:

|Field          |Description            |
----------------|-----------------------
|**PRODUCT_ID**| A unique identifier for each product.|
|**PRODUCT_NAME**| The name of the product, which typically includes the brand and the model.|
|**CATEGORY**| The general category of the product, which is "IT Accessories" for all entries sampled.|
|**DESCRIPTION**| A detailed description of the product, highlighting key features and specifications.|
|**UNIT_PRICE**| The price of the product in Euros.|
|**UNIT_MEASURE**| The unit of measure for the product, typically "Each" indicating pricing per item.|
|**SUPPLIER_ID**| A unique identifier for the supplier of the product.|
|**SUPPLIER_NAME**| The name of the supplier.|
|**LEAD_TIME_DAYS**| The number of days it takes from order to delivery.|
|**MIN_ORDER**| The minimum order quantity required.|
|**CURRENCY**| The currency of the transaction, which is "EURO" for all entries.|
|**SUPPLIER_COUNTRY**| The country where the supplier is located, which is "Germany" for all sampled entries.|
|**SUPPLIER_ADDRESS**| The physical address of the supplier.|
|**AVAILABILITY_DAYS**| The number of days the product is available for delivery.|
|**SUPPLIER_CITY**| The city where the supplier is located.|
|**STOCK_QUANTITY**| The quantity of the product currently in stock.|
|**MANUFACTURER**| The company that manufactured the product.|
|**CITY_LAT**| Geographical coordinates of the city (latitude)|
|**CITY_LONG**| Geographical coordinates of the city (longitude).|
|**RATING**| A rating for the product, which are on a scale from 1 to 5.|

</br>

This dataset is structured to support various business operations such as inventory management, order processing, and logistics planning, providing a comprehensive view of product offerings, supplier details, and stock levels. Each entry is highly detailed, suggesting the dataset could be used for analytical purposes, such as optimizing supply chain operations or analyzing sales and marketing strategies.

</br>


## Retrieval Augmented Generation using generative AI Hub and SAP HANA Vector Engine

### Hands-on Retrieval Augmented Generation (RAG) workflow 

The Retrieval Augmented Generation use case process consists of steps to be completed as seen in the graphic below. 

<br>


> ![title](./images/rag_full.png)

<br> 


#### Indexing Process
1. Business documents that should be used for answering user questions are fed into the model. The contents of the files are split into smaller chunks.
    >"Chunking" (and sometimes called "LLM chunking") refers to dividing a large text corpus into smaller, manageable pieces or segments. Each recursive chunking part acts as a standalone unit of information that can be individually indexed and retrieved. 
2. Embedding functions are used to create embeddings from the file/document chunks.
    >Embeddings refer to dense, continuous vectors representing text in a high-dimensional space. These vectors serve as coordinates in a semantic space, capturing the relationships and meanings between words.
3. The embeddings are then stored as vectors in the SAP HANA Cloud Database.
#### Retrieval Process
1. A query or prompt is submitted.
2. The query embedded into a vector form.
3. The query vector is compared to the values stored as vectors in SAP HANA Cloud via a similarity/semantic search.
4. The most appropriate and relevant results are identified.
5. And forwarded, along with the original query, to a large language model.
6. The LLM uses the results of the HANA vector search to augment its own searching capabilities, and the final answer is returned to the user.

### Implementing RAG Embeddings

- Prepare the documentation for the product catalog in CSV format with each row representing a product.
- Connect to the HANA vector storage instance and create a table to store the documentation data.
- Populate the table with data and create a REAL_VECTOR column to store embeddings.
- Use the Generative AI Hub SDK to define a function to generate embeddings for prompts and perform similarity search using the embeddings.

### Enhancing Query Responses

- Define a prompt template to provide context to queries.
- Modify the function to query the LLM (Large Language Model) based on the prompt template.
- Test the model's response using specific queries related to the node library, ensuring it provides contextually relevant responses based on embeddings.
  
> Retrieval augmented generation optimizes the output of large language models by applying more context to prompts.

## Setup and configuration

The following Python modules are to be installed during this hands-on introduction. 

#### **sap-ai-sdk-gen**

With this SAP python SDK you can leverage the power of generative Models like chatGPT available in SAP's generative AI Hub.

For more information, please see https://pypi.org/project/sap-ai-sdk-gen/

#### **hdbcli**

The Python Database API Specification v2.0 (PEP 249) defines a set of methods that provides a consistent database interface independent of the actual database being used. The Python extension module for SAP HANA implements PEP 249. Once you install the module, you can access and change the information in SAP HANA databases from Python.

For more information, please see https://pypi.org/project/hdbcli/

#### **langchain-hana**

Integrates LangChain with SAP HANA Cloud to make use of vector search, knowledge graph, and further in-database capabilities as part of LLM-driven applications.

For more information, please see https://pypi.org/project/langchain-hana/


</br>

#### Install Python packages

Run the following package installations. **pip** is the package installer for Python. You can use pip to install packages from the Python Package Index and other indexes.

In [ ]:
%pip install "sap-ai-sdk-gen[all]" --break-system-packages -U
%pip install hdbcli --break-system-packages -U
%pip install langchain-hana --break-system-packages -U

# kernel restart required!!!

#### Restart Python kernel

The Python kernel needs to be restarted before continuing. 

> ![title](./images/config_001.png)

#### Configure SAP Generative AI Hub credentials

Execute the configuration module below to enable access to SAP Generative AI foundation models. This configuration is automatically done by running configuration module in the code block.

You could also set up the same by running a terminal command: **aicore configure**


</br>

> Please ensure that the Python kernel was restarted!


In [ ]:
# Generative AI Config
import os
import json

with open(os.path.join(os.getcwd(), 'env_config.json')) as f:
    aicore_config = json.load(f)

### Initialize the LLM model
LLM is initialized with a model **anthropic--claude-4.5-sonnet**. This is used for generating responses or interacting in a chat-like environment.

> **IMPORTANT!** here you are connecting to the **anthropic--claude-4.5-sonnet** model that was deployed in SAP AI Core.
<!-- We can compare how the output produced by RAG is different from the output when we directly pass the prompt to the model. If we directly pass the prompt to the model without RAG, this will be the output. -->

In [ ]:
# Set llm
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from gen_ai_hub.proxy.gen_ai_hub_proxy import GenAIHubProxyClient
# harmonized init helper
from gen_ai_hub.proxy.langchain.init_models import init_llm

# Set up the AICoreV2Client
ai_core_client = AICoreV2Client(base_url=aicore_config['AICORE_BASE_URL'],
                            auth_url=aicore_config['AICORE_AUTH_URL'],
                            client_id=aicore_config['AICORE_CLIENT_ID'],
                            client_secret=aicore_config['AICORE_CLIENT_SECRET'],
                            resource_group=aicore_config['AICORE_RESOURCE_GROUP'])
# Set up the GenAIHubProxyClient
proxy_client = GenAIHubProxyClient(ai_core_client = ai_core_client)
print("✅AI Core Client connection is established successfully!")

# Set up the LLM model (here we are using Anthropic Claude 4.5 Sonnet as an example but you can choose any available model from here https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/README_sphynx.html#supported-models)
llm = init_llm('anthropic--claude-4.5-sonnet', proxy_client=proxy_client, max_tokens=800, temperature=0.3, top_p=None)
print("✅LLM model connection is established successfully!")

### Ask LLM without context

After completing the configuration we are ready to ask the first question directly to LLM (anthropic--claude-4.5-sonnet) without any business product context to find us products with a rating of 4 and more. The response is arbitrary and does not relate to our product data. 

</br>

> **Note** We can solve this problem by following the next steps in implementing RAG Embeddings.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from IPython.display import Markdown

# Define a simple prompt template for querying the LLM to get a keyboard suggestion based on the information from the Internet (for now).
TEMPLATE = """Advise the user based on the information available to you from general sources. Question: {question}"""

# Create a HumanMessagePromptTemplate using the defined template.
human_message_prompt = HumanMessagePromptTemplate.from_template(TEMPLATE)
# Create a ChatPromptTemplate using the human message prompt defined above.
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

question = "Return one keyboard suggestion (brand + model) with rating ≥4. Format as: 'Keyboard: <name>'"

# Format the prompt with the input question
prompt_text = chat_prompt.format_prompt(question=question).to_string()

# Invoke the LLM with the formatted prompt and get the response
llm_response = llm.invoke(prompt_text)

# Display the LLM response in a readable format
display(Markdown(llm_response.content.strip()))

# Implementing Retrieval Augmented Generation (RAG)

### Prepare the documentation for the product catalog in CSV format with each row representing a product

This code snippet demonstrates how to load and process text data from a CSV file using the `CSVLoader` from the `langchain_community.document_loaders` library.

This process is useful for handling large text data, making it more manageable or suitable for further processing, analysis, or input into machine learning models, especially when dealing with limitations on input size.

In [ ]:
from langchain_community.document_loaders import CSVLoader

FILE_PRODUCTS = "data/product_catalog.csv"

# Process CSV data file
loader = CSVLoader(
    file_path=FILE_PRODUCTS,
    source_column="PRODUCT_ID",
    metadata_columns=[
        "SUPPLIER_ID",
        "CATEGORY",
        "SUPPLIER_COUNTRY",
        "SUPPLIER_CITY",
        "MANUFACTURER",
    ],
    content_columns=[
        "PRODUCT_NAME",
        "DESCRIPTION",
        "UNIT_PRICE",
        "LEAD_TIME_DAYS",
        "STOCK_QUANTITY",
        "RATING",
        "MIN_ORDER",
        "CATEGORY",
        "SUPPLIER_NAME",
        "SUPPLIER_COUNTRY",
        "SUPPLIER_CITY",
        "SUPPLIER_ADDRESS",
        "STATUS",
        "CURRENCY",
        "MANUFACTURER",
        "CITY_LAT",
        "CITY_LONG",
    ],
    csv_args={"delimiter": ";", "quotechar": '"'},
    encoding="utf-8-sig",
)

# Process data
text_documents = loader.load()
print(f"📄 Loaded {len(text_documents)} documents")

for chunks in text_documents:
    print(chunks.metadata)
    print(chunks.page_content)

> At this point we have implemented the first RAG step - generated text chunks from source data
> 
> ![rag_indexing_1](./images/rag_indexing_1.png)

### SAP HANA Cloud vector engine

Storing vector embeddings within the same database is a strategic move that aligns seamlessly with SAP's commitment to providing a unified platform. This integration eliminates the hurdles posed by data silos, offering a holistic approach to data management. In SAP HANA Cloud, the storage of vector embeddings is seamlessly integrated into the platform's existing structure, allowing users to store them in a designated table. Developers can perform SQL-like queries effortlessly. 

This means you can execute joins, apply filters, and perform selects by combining vector embeddings with various data types, including transactional, spatial, graph, and JSON data, all within the same SQL environment. The Vector Engine ensures a user-friendly experience, eliminating the need for extensive learning or the adoption of new querying methodologies. Essentially, working with vector embeddings in SAP HANA Cloud is as straightforward as crafting queries in a standard SQL database, offering familiarity and ease of use for developers.

#### Connect to the HANA vector storage instance

The provided Python script imports database connection modules and initiates a connection to a SAP HANA Cloud instance using the `dbapi` module.

In [ ]:
# HC Vector Engine
import os
from hdbcli import dbapi

# Load HANA Cloud connection details
with open(os.path.join(os.getcwd(), 'env_cloud.json')) as f:
    hana_env_c = json.load(f)

# Establish a connection to the HANA Cloud database
connection = dbapi.connect( 
    address=hana_env_c['url'],
    port=hana_env_c['port'], 
    user=hana_env_c['user'], 
    password=hana_env_c['pwd']   
)

## Generate and store embeddings in SAP HANA Cloud

We will be using two types of Embedding Models to generate the embeddings: **text-embedding-3-large** from OpenAI and **SAP_GXY.20250407** from SAP.


### Initialize the OpenAI embedding model
Embeddings are vector representations of text data that incorporate the semantic meaning of the text. Define the embeddings object that generates embeddings from text data using the **text-embedding-3-large** model. This function will be used to generate embeddings from the user's prompts.

In [ ]:
# harmonized init helper
from gen_ai_hub.proxy.langchain.init_models import init_embedding_model
open_ai_embedding_model = init_embedding_model('text-embedding-3-large', proxy_client=proxy_client)

### Test the OpenAI embedding model

In [ ]:
# Input string
text = "SAP Generative AI Hub is awesome!"
# Generate the embedding
embedding = open_ai_embedding_model.embed_query(text)
print(embedding)

### Populate the table with data and create a REAL_VECTOR column to store embeddings

Create a LangChain VectorStore interface for the HANA database and specify the table (collection) to use for accessing the vector embeddings. Embeddings are vector representations of text data that incorporate the semantic meaning of the text.

The `langchain_hana` library, specifically the `HanaDB` class, from the LangChain community, enables interaction with SAP HANA Cloud's vector storage capabilities. It provides tools to manage and query vector embeddings stored in SAP HANA Cloud, making it easier to implement workflows like Retrieval Augmented Generation (RAG) using SAP HANA's advanced database features.

In [ ]:
from langchain_hana import HanaDB
# Create a LangChain VectorStore interface for the HANA database and specify the table (collection) to use for accessing the vector embeddings
db_openai_table = HanaDB(
    embedding=open_ai_embedding_model, 
    connection=connection, 
    table_name="PRODUCTS_IT_ACCESSORY_OPENAI_"+ hana_env_c['user'],
    content_column="VEC_TEXT", # the original text description of the product details
    metadata_column="VEC_META", # metadata associated with the product details
    vector_column="VEC_VECTOR" # the vector representation of each product 
)

In [ ]:
# Delete already existing documents from the table
db_openai_table.delete(filter={})

# add the loaded document chunks
db_openai_table.add_documents(text_documents)

### Verify product embeddings in SAP HANA Cloud

In [ ]:
# Query the table to verify embeddings
cursor = connection.cursor()
sql = f'SELECT VEC_TEXT, TO_NVARCHAR(VEC_VECTOR) FROM "{db_openai_table.table_name}"'

cursor.execute(sql)
vectors = cursor.fetchall()

for vector in vectors:
    print(vector)

> At this point we have implemented the Indexing Process part of RAG
> 
> ![rag_indexing_2](./images/rag_indexing_2.png)

## Performing a Vector Search to Find Relevant Products  

## Vector Search Using OpenAI's Embedding Model (*text-embedding-3-large*)

In this step, we use the **text-embedding-3-large** model to convert a natural language query into a vector representation and retrieve the most relevant records from a database using **vector similarity search**.  

In [ ]:
# Define the Query 
question = "Suggest a keyboard with a rating 4 or more"
# Using an embedding model (text-embedding-3-large), we transform the text query into a numerical vector
# Then, we perform a similarity search in the HANA database to find the most relevant product information based on the vector representation of the query. 
# The results are returned with their similarity scores, which indicate how closely they match the query.
results = db_openai_table.similarity_search_with_score(query=question, k=10)

for result in results:
    print(result)

## Vector Search Using SAP's Embedding Model (*SAP_GXY.20250407*)

You will:

1. **Create a table** in SAP HANA to store text data and generate vector embeddings using **SAP_GXY.20250407**.  
2. **Insert product descriptions** into the table, allowing automatic embedding generation.  
3. **Perform a vector search** to retrieve the most relevant products based on semantic similarity.  


### Creating a table with automatic vector embeddings 

In [ ]:
from langchain_hana import HanaInternalEmbeddings

# HANA-internal embedding
internal_emb = HanaInternalEmbeddings(internal_embedding_model_id="SAP_GXY.20250407")

db_internal_emb_table = HanaDB(
    embedding=internal_emb, 
    connection=connection, 
    table_name="PRODUCTS_IT_ACCESSORY_SAP_"+ hana_env_c['user'],
    content_column="VEC_TEXT", # the original text description of the product details
    metadata_column="VEC_META", # metadata associated with the product details
    vector_column="VEC_VECTOR" # the vector representation of each product 
)

### Inserting data in SAP HANA Cloud

In [ ]:
# Delete already existing documents from the table
db_internal_emb_table.delete(filter={})

# add the loaded document chunks
db_internal_emb_table.add_documents(text_documents)

### Querying the table to verify stored embeddings

In [ ]:
# Query the table to verify embeddings
cursor = connection.cursor()
sql = f'SELECT VEC_TEXT, TO_NVARCHAR(VEC_VECTOR) FROM "{db_internal_emb_table.table_name}"'

cursor.execute(sql)
vectors = cursor.fetchall()

for vector in vectors:
    print(vector)

### Perform a vector search using SAP's embedding model

- Using an internal SAP HANA embedding model `SAP_GXY.20250407` we transform the text query into a numerical vector
- Then, we perform a similarity search in the HANA database to find the most relevant product information based on the vector representation of the query. 
- The results are returned with their similarity scores, which indicate how closely they match the query.

In [ ]:
question = "Suggest a keyboard with a rating 4 or more"

results = db_internal_emb_table.similarity_search_with_score(query=question, k=10)

for result in results:
    print(result)

> At this point we have successfully implemented the first step of Retrieval Process and enabled semantic vector similarity search to **retrieve relevant results from SAP HANA Cloud** based on user question.
> 
> ![rag_retrieval_1](./images/rag_retrieval_1.png) 

## Enhancing Query Responses by Passing Context and Prompt to LLM

### Define a prompt template to provide context to queries

Define a prompt template to provide context to our prompts. Thus, when passed to the model, the template will add the necessary context to the prompt so that more accurate results are generated.

> The created template for the prompt contains two variables - **`context`** and **`query`**. These variables will be replaced with the context and question in the upcoming steps.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

# Define the prompt template for product recommendation using retrieved context from HANA database.
# This prompt guides the LLM to provide accurate and relevant product recommendations based solely on the information retrieved from the database, without making any assumptions or fabricating details. The prompt emphasizes the importance of adhering to the provided context and clearly outlines the expected response format and behavior in various scenarios (e.g., when no products match the criteria or when information is incomplete).
PRODUCT_RAG_PROMPT = """
    You are a product recommendation assistant.

    Your job is to help users find and understand products using ONLY the information provided in the retrieved context.

    You must follow these rules strictly:

    --------------------------------------------------
    1. Use only retrieved context
    --------------------------------------------------
    - Base your answer only on the provided context.
    - Never invent product names, specifications, ratings, or prices.
    - If required information is missing, say you don’t know.

    --------------------------------------------------
    2. Understand user intent
    --------------------------------------------------
    The user may ask to:
    - recommend products
    - filter products by criteria (rating, price, category, brand, features)
    - compare products
    - explain product features
    - summarize options
    - find best match for a need

    Interpret the request and use the context to respond appropriately.

    --------------------------------------------------
    3. When recommending products
    --------------------------------------------------
    If matching products exist:
    - return only products that meet the criteria
    - clearly list product name and relevant attributes
    - explain briefly why each product matches

    If no products match:
    Respond exactly:
    "I could not find any products that match your criteria."

    --------------------------------------------------
    4. When information is incomplete
    --------------------------------------------------
    If the context does not contain enough information:
    Respond exactly:
    "I don’t have enough information to answer that."

    --------------------------------------------------
    5. Do not expose system details
    --------------------------------------------------
    Never mention:
    - embeddings
    - vector search
    - retrieval
    - metadata
    - internal processing

    --------------------------------------------------
    6. Response style
    --------------------------------------------------
    - Be clear and concise
    - Use structured lists when helpful
    - Be factual and neutral
    - Do not speculate

    --------------------------------------------------

    User question:
    {query}

    Context:
    {context}
    """

# Create a HumanMessagePromptTemplate using the defined prompt template. 
human_message_prompt = HumanMessagePromptTemplate.from_template(PRODUCT_RAG_PROMPT)

# Create a ChatPromptTemplate using the human message prompt defined above. 
# This template will be used to format the input query and the retrieved context from the HANA database when invoking the LLM for generating product recommendations based on the retrieved information.
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

### Complex Query and Contextual Product Recommendation

- This code block demonstrates how to use a vector similarity search to retrieve the most relevant product information from SAP HANA Cloud based on a user query with specific criteria (e.g., rating, availability, delivery time).
- The retrieved context is then formatted and passed to a Large Language Model (LLM), which generates product recommendations strictly grounded in the actual data, ensuring accurate and context-aware responses
- The approach combines semantic search and generative AI to deliver precise, criteria-based product suggestions, avoiding fabricated or irrelevant answers.

In [ ]:
# Define a more complex query that requires the LLM to utilize the retrieved context effectively to provide accurate and relevant product recommendations based on specific criteria such as rating, availability, and delivery time.
question = "Suggest keyboards with a rating 4 or more. Make sure that the suggested products are available for an order and can be delivered in 10 days or less."

# Using an embedding model (text-embedding-3-large), we transform the text query into a numerical vector
# Then, we perform a similarity search in the HANA database to find the most relevant product information based on the vector representation of the query.
# The results are returned with their similarity scores, which indicate how closely they match the query.
vector_search_results = db_openai_table.similarity_search_with_score(
    query=question, k=10
)

# We extract the relevant context from the search results to provide it as input to the LLM for generating a response that is grounded in the retrieved information from the HANA database.
context = "\n\n".join(doc.page_content for doc, _ in results)

# Finally, we format the prompt with the user query and the retrieved context to create the final input that will be sent to the LLM for generating product recommendations based on the information retrieved from the HANA database.
prompt_text = chat_prompt.format_prompt(query=question, context=context).to_string()

# We invoke the LLM with the formatted prompt to generate a response that provides product recommendations based on the retrieved context from the HANA database, adhering to the guidelines specified in the prompt template.
llm_response = llm.invoke(prompt_text)

print("💬LLM Response:")
display(Markdown(llm_response.content.strip()))

print("\n\n💯This is the end of the Jupiter Notebook. Thank you for your attention!")
print("📸 Please take a screenshot!")

> At this point we have successfully implemented the final step of Retrieval Process and enabled generation of the **contextually relevant answer** based on user's query.
>
> ![rag_full](./images/rag_full.png) 

### Close Database Connection
Ensure the database connection is closed when the notebook is no longer in use.

In [ ]:
if cursor:
    cursor.close()
if connection:
    connection.close()
print("Database connection closed.")